# JSON Serialization with lionherd_core.ln

Fast, extensible JSON serialization built on orjson with production-grade defaults.

**Core Features:**
- **High Performance**: orjson-based (C/Rust backend)
- **Extended Type Support**: Path, Decimal, Enum, sets, datetime, UUID
- **Custom Serializers**: Extensible default() factory
- **Safe Fallback**: Never-fail mode for logging
- **NDJSON Streaming**: Memory-efficient line-by-line output
- **Flexible Options**: Pretty-print, key sorting, datetime formats

In [1]:
import datetime as dt
import decimal
from enum import Enum
from pathlib import Path
from uuid import uuid4

from lionherd_core.ln import (
    get_orjson_default,
    json_dumpb,
    json_dumps,
    json_lines_iter,
)

## 1. Basic Serialization

Two main functions:
- `json_dumps()`: Returns str (default) or bytes
- `json_dumpb()`: Returns bytes (faster, prefer in hot code)

In [2]:
# Basic data
data = {"name": "Alice", "age": 30, "active": True, "scores": [95, 87, 92]}

# json_dumps - returns str by default
result_str = json_dumps(data)
print(f"Type: {type(result_str).__name__}")
print(f"Result: {result_str}")

Type: str
Result: {"name":"Alice","age":30,"active":true,"scores":[95,87,92]}


In [3]:
# json_dumpb - returns bytes (fast path)
result_bytes = json_dumpb(data)
print(f"Type: {type(result_bytes).__name__}")
print(f"Result: {result_bytes}")

# Decode if needed
print(f"Decoded: {result_bytes.decode('utf-8')}")

Type: bytes
Result: b'{"name":"Alice","age":30,"active":true,"scores":[95,87,92]}'
Decoded: {"name":"Alice","age":30,"active":true,"scores":[95,87,92]}


## 2. Extended Type Support

Built-in serialization for Python types not in standard JSON:
- **Path**: Serialized as strings
- **Decimal**: As float (fast) or str (precise)
- **Enum**: As name or value
- **set/frozenset**: As lists (optionally sorted)
- **datetime/date/time/UUID**: Native orjson support

In [4]:
# Define test enum
class Status(Enum):
    PENDING = 1
    ACTIVE = 2
    COMPLETE = 3


# Extended types
extended_data = {
    "path": Path("/tmp/data.json"),
    "amount": decimal.Decimal("123.456789"),
    "status": Status.ACTIVE,
    "tags": {"python", "json", "serialization"},
    "id": uuid4(),
    "created": dt.datetime(2025, 1, 1, 12, 0, 0, tzinfo=dt.UTC),
}

# Default: Decimal as float, Enum as value
result = json_dumps(extended_data, pretty=True)
print("Default serialization:")
print(result)

Default serialization:
{
  "path": "/tmp/data.json",
  "amount": "123.456789",
  "status": 2,
  "tags": [
    "json",
    "python",
    "serialization"
  ],
  "id": "c354fbe9-61dc-4f0d-ac9f-8a8112995a3b",
  "created": "2025-01-01T12:00:00+00:00"
}


In [5]:
# Decimal as string (preserve precision)
result = json_dumps(extended_data, decimal_as_float=False, pretty=True)
print("Decimal as string:")
print(result)

Decimal as string:
{
  "path": "/tmp/data.json",
  "amount": "123.456789",
  "status": 2,
  "tags": [
    "json",
    "python",
    "serialization"
  ],
  "id": "c354fbe9-61dc-4f0d-ac9f-8a8112995a3b",
  "created": "2025-01-01T12:00:00+00:00"
}


In [6]:
# Enum as name
result = json_dumps(extended_data, enum_as_name=True, pretty=True)
print("Enum as name:")
print(result)

Enum as name:
{
  "path": "/tmp/data.json",
  "amount": "123.456789",
  "status": 2,
  "tags": [
    "json",
    "python",
    "serialization"
  ],
  "id": "c354fbe9-61dc-4f0d-ac9f-8a8112995a3b",
  "created": "2025-01-01T12:00:00+00:00"
}


In [7]:
# Deterministic set ordering (stable across runs)
result = json_dumps(extended_data, deterministic_sets=True, pretty=True)
print("Deterministic sets:")
print(result)

Deterministic sets:
{
  "path": "/tmp/data.json",
  "amount": "123.456789",
  "status": 2,
  "tags": [
    "json",
    "python",
    "serialization"
  ],
  "id": "c354fbe9-61dc-4f0d-ac9f-8a8112995a3b",
  "created": "2025-01-01T12:00:00+00:00"
}


## 3. Options Control

Fine-grained control via `make_options()` or kwargs:
- **pretty**: Indent with 2 spaces
- **sort_keys**: Alphabetical key ordering
- **naive_utc**: Treat naive datetimes as UTC
- **utc_z**: Use 'Z' suffix instead of '+00:00'
- **append_newline**: Add trailing newline
- **allow_non_str_keys**: Allow int/bool keys (non-standard JSON)

In [8]:
data = {"zebra": 1, "apple": 2, "mango": 3}

# Pretty-print with sorted keys
result = json_dumps(data, pretty=True, sort_keys=True)
print("Pretty + sorted:")
print(result)

Pretty + sorted:
{
  "apple": 2,
  "mango": 3,
  "zebra": 1
}


In [9]:
# Datetime formatting
dt_data = {"timestamp": dt.datetime(2025, 1, 1, 12, 0, 0, tzinfo=dt.UTC)}

# Default: '+00:00' suffix
print("Default:")
print(json_dumps(dt_data))

# UTC 'Z' suffix
print("\nWith utc_z:")
print(json_dumps(dt_data, utc_z=True))

Default:
{"timestamp":"2025-01-01T12:00:00+00:00"}

With utc_z:
{"timestamp":"2025-01-01T12:00:00Z"}


In [ ]:
# Non-string keys (non-standard JSON, but useful)
non_str_keys = {1: "one", 2: "two", 3: "three"}

result = json_dumps(non_str_keys, allow_non_str_keys=True)
print("Non-string keys:")
print(result)

## 4. Custom Serializers

Build custom `default=` functions via `get_orjson_default()`:
- Define custom type handlers
- Control serialization order
- Extend or replace defaults

In [11]:
# Custom class
class Person:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age


# Custom serializer
def serialize_person(p: Person) -> dict:
    return {"name": p.name, "age": p.age, "type": "Person"}


# Build default function with Person in order
custom_default = get_orjson_default(
    order=[Person],  # Check Person type
    additional={Person: serialize_person},
    extend_default=True,  # Keep built-in serializers
)

# Use custom default
data = {"person": Person("Bob", 25), "created": dt.datetime.now(dt.UTC)}
result = json_dumps(data, default=custom_default, pretty=True)
print(result)

{
  "person": {
    "name": "Bob",
    "age": 25,
    "type": "Person"
  },
  "created": "2025-11-09T15:36:22.402102+00:00"
}


In [12]:
# Pydantic model support (duck-typed)
from pydantic import BaseModel


class User(BaseModel):
    name: str
    email: str
    age: int


user = User(name="Charlie", email="charlie@example.com", age=35)

# Automatically calls model_dump()
result = json_dumps({"user": user}, pretty=True)
print("Pydantic model (auto model_dump()):")
print(result)

Pydantic model (auto model_dump()):
{
  "user": {
    "name": "Charlie",
    "email": "charlie@example.com",
    "age": 35
  }
}


## 5. Safe Fallback Mode

Never-fail serialization for logging (use with caution):
- Unknown types → clipped repr()
- Exceptions → {"type": ..., "message": ...}
- **Recommended for logging ONLY** (lossy)

In [13]:
# Custom class with no serializer
class Widget:
    def __init__(self, name: str):
        self.name = name

    def __repr__(self):
        return f"Widget(name={self.name!r})"


# Without safe_fallback - raises TypeError
try:
    json_dumps({"widget": Widget("gear")})
except TypeError as e:
    print(f"Error: {e}")

Error: Type is not JSON serializable: Widget


In [14]:
# With safe_fallback - serializes repr()
result = json_dumps({"widget": Widget("gear")}, safe_fallback=True, pretty=True)
print("Safe fallback (repr):")
print(result)

Safe fallback (repr):
{
  "widget": "Widget(name='gear')"
}


In [15]:
# Exception serialization
try:
    raise ValueError("Something went wrong")
except ValueError as e:
    log_data = {"error": e, "timestamp": dt.datetime.now(dt.UTC)}
    result = json_dumps(log_data, safe_fallback=True, pretty=True)
    print("Exception in safe mode:")
    print(result)

Exception in safe mode:
{
  "error": {
    "type": "ValueError",
    "message": "Something went wrong"
  },
  "timestamp": "2025-11-09T15:36:22.416594+00:00"
}


In [16]:
# Fallback clipping (long repr)
long_repr = "x" * 5000


class LongWidget:
    def __repr__(self):
        return long_repr


result = json_dumps(
    {"widget": LongWidget()},
    safe_fallback=True,
    fallback_clip=100,  # Clip at 100 chars
)
print(f"Clipped length: {len(result)}")
print(result)

Clipped length: 29
{"widget":"...(+4900 chars)"}


## 6. NDJSON Streaming

Memory-efficient line-by-line JSON for large datasets:
- One JSON object per line
- Returns bytes iterable
- Configurable serialization options

In [17]:
# Generate stream of records
records = [
    {"id": 1, "name": "Alice", "score": 95},
    {"id": 2, "name": "Bob", "score": 87},
    {"id": 3, "name": "Charlie", "score": 92},
]

# Stream as NDJSON
lines = list(json_lines_iter(records))
print(f"Generated {len(lines)} lines\n")

for line in lines:
    print(f"Type: {type(line).__name__}, Content: {line.decode('utf-8')}", end="")

Generated 3 lines

Type: bytes, Content: {"id":1,"name":"Alice","score":95}
Type: bytes, Content: {"id":2,"name":"Bob","score":87}
Type: bytes, Content: {"id":3,"name":"Charlie","score":92}


In [18]:
# Extended types in streaming
stream_data = [
    {"id": uuid4(), "created": dt.datetime.now(dt.UTC), "amount": decimal.Decimal("99.99")},
    {"id": uuid4(), "created": dt.datetime.now(dt.UTC), "amount": decimal.Decimal("199.50")},
]

for line in json_lines_iter(stream_data, decimal_as_float=False, utc_z=True):
    print(line.decode("utf-8"), end="")

{"id":"b51f2594-014e-44f7-8805-14473c5357fd","created":"2025-11-09T15:36:22.427034Z","amount":"99.99"}
{"id":"1dcc9d84-cf67-4bd3-9336-4fc579e2e6ba","created":"2025-11-09T15:36:22.427164Z","amount":"199.50"}


## 7. Performance Comparison

Compare str vs bytes output performance (bytes is faster).

In [19]:
import timeit

# Test data
test_data = {
    "users": [
        {"id": i, "name": f"User{i}", "active": i % 2 == 0, "score": i * 10.5} for i in range(100)
    ]
}

# Benchmark
n = 1000

time_bytes = timeit.timeit(lambda: json_dumpb(test_data), number=n)

time_str = timeit.timeit(lambda: json_dumps(test_data), number=n)

print(f"json_dumpb (bytes): {time_bytes * 1000:.2f}ms for {n} iterations")
print(f"json_dumps (str):   {time_str * 1000:.2f}ms for {n} iterations")
print(f"Speedup: {time_str / time_bytes:.2f}x")

json_dumpb (bytes): 8.75ms for 1000 iterations
json_dumps (str):   10.06ms for 1000 iterations
Speedup: 1.15x


## 8. Advanced: Custom Default Order

Control type-checking order for performance optimization.

In [20]:
# Custom type order (check most common types first)
custom_default = get_orjson_default(
    order=[decimal.Decimal, Path, set],  # Check these first
    extend_default=True,  # Then built-in types
    decimal_as_float=True,
    deterministic_sets=True,
)

data = {
    "price": decimal.Decimal("49.99"),
    "path": Path("/data/output.json"),
    "tags": {"fast", "reliable", "tested"},
}

result = json_dumps(data, default=custom_default, pretty=True)
print(result)

{
  "price": 49.99,
  "path": "/data/output.json",
  "tags": [
    "fast",
    "reliable",
    "tested"
  ]
}


## Summary Checklist

**lionherd_core.ln JSON Utilities:**
- ✅ High-performance orjson backend (C/Rust)
- ✅ Extended types: Path, Decimal, Enum, set, datetime, UUID
- ✅ Flexible serialization modes (str/bytes)
- ✅ Custom serializers via `get_orjson_default()`
- ✅ Rich options: pretty, sort_keys, datetime formats
- ✅ Safe fallback for logging (never-fail)
- ✅ NDJSON streaming for large datasets
- ✅ Pydantic model auto-detection (model_dump)
- ✅ Deterministic set ordering
- ✅ Configurable repr clipping

**Best Practices:**
- Use `json_dumpb()` in hot code paths (bytes faster than str)
- Use `safe_fallback=True` only for logging (lossy)
- Use `deterministic_sets=True` for reproducible output (testing)
- Use `json_lines_iter()` for large datasets (memory-efficient)
- Extend defaults with `additional=` rather than replacing

**Next Steps:**
- See `ln.json_load` for deserialization
- See `Element.to_json()` for integration examples